# Курсовая работа <br> Прогнозирование качества вина

Выполнила: Кузнецова Екатерина

## 1. Цель работы

Улучшить практические навыки работы с различными методами машинного обучения с использованием языка программирования python.

## 2. Постановка задачи

1. Выбрать соревнование на сайте kaggle.com.
2. Прочитать описание задачи и скачать приложенные к заданию датасеты.
3. С успехом выполнить выбранную задачу.
4. Отправить данные на kaggle.com.

## 3. Исходные данные

На kaggle.com размещено несколько сотен интереснейших соревнований, касающихся различных сфер жизни человека. Некоторые из них доступны только ограниченному кругу лиц, а в остальных может поучаствовать любой желающий. К последним относится и выбранная мною задача, "Aalto Wine Quality Prediction". Её авторы задались целью узнать, что делает вино хорошим. То есть какие показатели определяют качество этого благородного (или не очень) напитка.

Соревнование: https://inclass.kaggle.com/c/aalto-wine-quality-prediction-t-61-3050-challenge<br>
Предметная область: качественные характеристики вин<br>
Задача: научиться прогнозировать качество вин без рейтинга экспертов

К заданию было приложено 3 файла:

- training_classification_regression_2015.csv (тренировочный набор данных)
- challenge_public_test_classification_regression.csv (тестовый набор данных)
- challenge_regression_sample_submission.csv (формат представления ответа)

В каждом файле содержатся наименования атрибутов и их значения.

#### Тренировочный датасет:

Количество записей: 5000<br>
Количество атрибутов: 13


Атрибуты:

1. fixedAcidity (float)
2. volatileAcidity (float)
3. citricAcid (float)
4. residualSugar (float)
5. chlorides (float)
6. freeSulfurDioxide (float)
7. totalSulfurDioxide (float)
8. density (float)
9. pH (float)
10. alcohol (float)
11. quality (целочисленные значения в диапазоне [1,7])
12. type (строковое значение из набора {'Red', 'White'})

#### Тестовый датасет:

Количество записей: 1000<br>
Количество атрибутов: 14

Тестовый датасет содержит также поле id (int)

#### Формат представления ответа:

Атрибуты:

1. id (int)
2. quality (целочисленные значения в диапазоне [1,7])

## 4. Ход работы

#### 1. Загрузка и анализ данных

Загружаем обучающий набор данных. Как мы видим, в нём нет пропущенных значений, почти все значения относятся к вещественному типу (значит нормализация этих данных не будет лишней).

In [1]:
import pandas as pd
train_df = pd.read_csv('alc/training_classification_regression_2015.csv', header=0)
print(train_df[:7])
print('Количество записей в тренировочном датасете: %d, количество атрибутов: %d' % train_df.shape)

   fixedAcidity  volatileAcidity  citricAcid  residualSugar  chlorides  \
0      6.945513         0.272992    0.403691      13.920038   0.051043   
1      6.995486         0.190402    0.310377      19.186267   0.044147   
2      5.978831         0.140616    0.251393       4.402912   0.028109   
3      6.688603         0.371501    0.507942      11.862843   0.044918   
4      7.224421         0.201252    0.220918       1.612570   0.044984   
5     13.386383         0.460975    0.518178       2.115102   0.071775   
6      7.065214         0.541150    0.090961       2.028085   0.082078   

   freeSulfurDioxide  totalSulfurDioxide   density        pH  sulphates  \
0          66.077662          245.612070  0.998768  3.161269   0.579738   
1          39.852796          175.908433  1.000245  2.932994   0.523323   
2          32.093612          151.569367  0.995412  3.488070   0.511582   
3          67.531739          155.608503  0.998803  3.174425   0.439773   
4          16.817987          12

Посмотрим теперь, что содержится в тестовой выборке. Среди значений основных атрибутов также нет пропусков, но для всех позиций не определены параметры "качество" и "тип" вина.

In [2]:
test_df = pd.read_csv('alc/challenge_public_test_classification_regression_2015.csv', header=0)
print(test_df[:7])
print('Количество записей в тестовом датасете: %d, количество атрибутов: %d' % test_df.shape)

   id  fixedAcidity  volatileAcidity  citricAcid  residualSugar  chlorides  \
0   1      6.623879         0.350308    0.291192      14.231505   0.044939   
1   2      7.001997         0.360802    0.251197       5.802122   0.016012   
2   3      8.365224         0.199237    0.313605       2.790155   0.054998   
3   4      7.206049         0.262821    0.300107       9.007575   0.051003   
4   5      6.915703         0.338247    0.735664      11.227664   0.069907   
5   6      8.444906         0.563366    0.081195       2.098270   0.106025   
6   7      7.873634         0.220525    0.419400      14.503407   0.044964   

   freeSulfurDioxide  totalSulfurDioxide   density        pH  sulphates  \
0          56.313779          173.311310  1.000947  3.184398   0.579831   
1          14.184137           66.170046  0.990834  2.847829   0.593433   
2          16.875336           83.105477  0.994815  2.959980   0.451726   
3          33.906743          169.741299  0.996002  3.217775   0.496368   


Так как в тестовой выборке не указан тип вина, следовательно для определения качества напитка он не так важен. Поэтому уберем этот параметр из обеих выборок.
Также уберем из тестовой выборки поля "id" и "quality", которое нужно предсказать.

In [3]:
# разбиение обучающего датасета на значения атрибутов и массив классов
train_wine_attr = train_df.drop(['quality', 'type'], axis=1).values
train_wine_quality = train_df['quality'].astype(int).values

test_wine_attr = test_df.drop(['id', 'quality', 'type'], axis=1).values

Нормализуем вещественные атрибуты для их более удобной классификации и сравнения.

In [4]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(train_wine_attr)
train_wine_attr = scaler.transform(train_wine_attr)
test_wine_attr = scaler.transform(test_wine_attr)

В итоге мы получем функции, которые загружают и подготавливают обучающие и тестовые данные для их дальнейшей классификации.

In [5]:
def load_train_data():
    train_df = pd.read_csv('training_classification_regression_2015.csv', header=0)
    # print('Количество записей в тренировочном датасете: %d, количество атрибутов: %d' % train_df.shape)

    # разбиение обучающего датасета на значения атрибутов и массив классов
    train_wine_attr = train_df.drop(['quality', 'type'], axis=1).values
    train_wine_quality = train_df['quality'].astype(int).values

    # нормализация значений атрибутов
    scaler = preprocessing.StandardScaler().fit(train_wine_attr)
    train_wine_attr = scaler.transform(train_wine_attr)

    return train_wine_attr, train_wine_quality, scaler


def load_test_data(scaler):
    test_df = pd.read_csv('challenge_public_test_classification_regression_2015.csv', header=0)
    # print('Количество записей в тестовом датасете: %d, количество атрибутов: %d' % test_df.shape)

    # удаление ненужных атрибутов тестового массива
    test_wine_attr = test_df.drop(['id', 'quality', 'type'], axis=1).values

    # нормализация значений атрибутов
    test_wine_attr = scaler.transform(test_wine_attr)

    return test_wine_attr, test_df['id']

#### 2. Подбор классификатора

Так как множество значений атрибута "качество" конечно и имеет 7 целочисленных значений, следовательно мы решаем задачу классификации. Для обработки полученных данных и предсказания оценки качества каждого напитка из списка, необходимо выбрать подходящую модель классификатора. В ходе выполнения лабораторных работ было выявлено, что собственноручно реализованные алгоритмы работаю несколько хуже, нежели аналогичные методы, содержащиеся в библиотеке scikit learn. Поэтому было принято решение использовать для классификации библиотечную реализацию алгоритма. В scikit learn содержится более 20 моделей классификаторов, работающих с разной степенью точности. Для достижения оптимальных результатов, протестируем несколько моделей.

В работе сравниваются следующие классификаторы:

- DummyClassifier
- AdaBoostClassifier
- BaggingClassifier
- ExtraTreesClassifier
- GradientBoostingClassifier
- RandomForestClassifier
- LogisticRegression
- PassiveAggressiveClassifier
- RidgeClassifier
- SGDClassifier
- GaussianNB
- KNeighborsClassifier
- NearestCentroid
- MLPClassifier
- LabelPropagation
- SVC
- LinearSVC
- DecisionTreeClassifier
- ExtraTreeClassifier

Используем каждый из алгоритмов на наших данных и выберем те, которые классифицируют тренировочный датасет лучше. Для более точных результатов будем использовать кросс-валидацию.

In [6]:
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.neural_network import MLPClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier

import warnings
warnings.filterwarnings("ignore")

def test_class_models(train_wine_attr, train_wine_quality):
    class_models = [DummyClassifier(), AdaBoostClassifier(), BaggingClassifier(), ExtraTreesClassifier(),
                    GradientBoostingClassifier(), RandomForestClassifier(), LogisticRegression(),
                    PassiveAggressiveClassifier(), RidgeClassifier(), SGDClassifier(), GaussianNB(),
                    KNeighborsClassifier(), NearestCentroid(), MLPClassifier(),
                    LabelPropagation(), SVC(), LinearSVC(), DecisionTreeClassifier(),
                    ExtraTreeClassifier()]

    for clf in class_models:
        result = cross_val_score(clf, train_wine_attr, train_wine_quality, cv=KFold(n_splits=50))
        print('Метод классификации: {}.\n  Точность: {:.5f} ({:.5f})'.format(clf, 
                                                                             result.mean(), 
                                                                             result.std()))
        
test_class_models(train_wine_attr, train_wine_quality)

Метод классификации: DummyClassifier(constant=None, random_state=None, strategy='stratified').
  Точность: 0.33400 (0.04285)


Метод классификации: AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None).
  Точность: 0.24140 (0.08530)


Метод классификации: BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
         verbose=0, warm_start=False).
  Точность: 0.59220 (0.04415)


Метод классификации: ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False).
  Точность: 0.62800 (0.04972)


Метод классификации: GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False).
  Точность: 0.56480 (0.05262)


...

Метод классификации: LabelPropagation(alpha=1, gamma=20, kernel='rbf', max_iter=30, n_jobs=1,
         n_neighbors=7, tol=0.001).
  Точность: 0.63100 (0.05255)


Метод классификации: SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False).
  Точность: 0.57180 (0.04629)


Метод классификации: LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0).
  Точность: 0.52840 (0.04917)


Метод классификации: DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best').
  Точность: 0.52860 (0.06416)


Метод классификации: ExtraTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_split=1e-07, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          random_state=None, splitter='random').
  Точность: 0.50520 (0.04267)


Самая высокая точность классификации получилась при использовании методов RandomForestClassifier, ExtraTreesClassifier и LabelPropagation.

Так как классификаторы запускались без параметров, то есть шанс улучшить показатели точности путем подбора наиболее удачных значений для атрибутов.

Исследование параметров LabelPropagation.

In [7]:
def test_LabelPropagation(train_wine_attr, train_wine_quality):
    print('Тестирование LabelPropagation')

    # Без параметров
    result = cross_val_score(LabelPropagation(), train_wine_attr, train_wine_quality, cv=KFold(n_splits=20))
    print('Точность без параметров: {:.5f} ({:.5f})'.format(result.mean(), result.std()))

    # Функция ядра
    # knn
    for i in [10, 20, 30, 50, 60]:
        res = cross_val_score(LabelPropagation(kernel='knn', n_neighbors=i), 
                              train_wine_attr, train_wine_quality,
                              cv=KFold(n_splits=20))
        print('kernel = knn, n_neighbors =', i, res.mean(), res.std())
    # rbf
    res = cross_val_score(LabelPropagation(kernel='rbf'), 
                          train_wine_attr, train_wine_quality,
                          cv=KFold(n_splits=20))
    print('kernel = rbf', res.mean(), res.std())
    print('--------------')

    # Параметр для rbf ядра
    gammas = [0.1, 0.5, 1, 5, 10, 20, 30, 40]
    for gamma in gammas:
        res = cross_val_score(LabelPropagation(kernel='rbf', gamma=gamma), 
                              train_wine_attr, train_wine_quality,
                              cv=KFold(n_splits=20))
        print('gamma =', gamma, res.mean(), res.std())
    print('--------------')

    # Параметр для rbf ядра
    alphas = [0.01, 0.1, 0.5, 1, 2, 3]
    for alpha in alphas:
        res = cross_val_score(LabelPropagation(alpha=alpha), 
                              train_wine_attr, train_wine_quality,
                              cv=KFold(n_splits=20))
        print('alpha =', alpha, res.mean(), res.std())
    print('--------------')
    
test_LabelPropagation(train_wine_attr, train_wine_quality)

Тестирование LabelPropagation


Точность без параметров: 0.62540 (0.02702)


kernel = knn, n_neighbors = 10 0.495 0.0257021399887


kernel = knn, n_neighbors = 20 0.4762 0.0295154196989


kernel = knn, n_neighbors = 30 0.4776 0.0321347164294


...

alpha = 0.1 0.624 0.0281424945589


alpha = 0.5 0.624 0.0281424945589


alpha = 1 0.6254 0.0270192523953


alpha = 2 0.6236 0.0282814426789


alpha = 3 0.6238 0.0282127630692
--------------


Исследование параметров ExtraTreesClassifier.

In [11]:
import numpy as np


def test_ExtraTreesClassifier(train_wine_attr, train_wine_quality):
    print('Тестирование ExtraTreesClassifier')

    # Без параметров
    result = cross_val_score(ExtraTreesClassifier(), train_wine_attr, train_wine_quality, cv=KFold(n_splits=20))
    print('Точность без параметров: {:.5f} ({:.5f})'.format(result.mean(), result.std()))

    # Количество деревьев (default=10)
    n_estimators = np.arange(10, 100, 10)
    for n_estimator in n_estimators:
        result = cross_val_score(ExtraTreesClassifier(n_estimators=n_estimator), train_wine_attr, train_wine_quality,
                                 cv=KFold(n_splits=20))
        print('n_estimators =', n_estimator, result.mean(), result.std())
    print('--------------')

    # Функция оценки качества разбиения (default=”gini”)
    criterion = ['gini', 'entropy']
    for cr in criterion:
        result = cross_val_score(ExtraTreesClassifier(criterion=cr), 
                                 train_wine_attr, train_wine_quality,
                                 cv=KFold(n_splits=20))
        print('criterion =', cr, result.mean(), result.std())
    print('--------------')

    # Количество фич при поиске наилучшего разбиения (default=”auto”)
    max_features = ['auto', 'log2', 1, 3, 5, 7, 9, 11]
    for features in max_features:
        result = cross_val_score(ExtraTreesClassifier(max_features=features), 
                                 train_wine_attr, train_wine_quality,
                                 cv=KFold(n_splits=20))
        print('max_features =', features, result.mean(), result.std())
    print('--------------')

    # Использовать ли предыдушие решения? (default=False)
    warm_start = [True, False]
    for war_start in warm_start:
        result = cross_val_score(ExtraTreesClassifier(warm_start=war_start), 
                                 train_wine_attr, train_wine_quality,
                                 cv=KFold(n_splits=20))
        print('warm_start =', war_start, result.mean(), result.std())
    print('--------------')
    
test_ExtraTreesClassifier(train_wine_attr, train_wine_quality)

Тестирование ExtraTreesClassifier


Точность без параметров: 0.62360 (0.02629)


n_estimators = 10 0.625 0.0319718626295


n_estimators = 20 0.6536 0.0247030362506


n_estimators = 30 0.6596 0.0277964026449


...

max_features = 7 0.622 0.0278136657059


max_features = 9 0.6242 0.029650632371


max_features = 11 0.6256 0.026027677576
--------------


warm_start = True 0.6272 0.0225424044858


warm_start = False 0.6288 0.0324863048068
--------------


Исследование параметров RandomForestClassifier.

In [13]:
def test_RandomForestClassifier(train_wine_attr, train_wine_quality):
    print('Тестирование RandomForestClassifier')

    # Без параметров
    result = cross_val_score(RandomForestClassifier(), 
                             train_wine_attr, train_wine_quality, 
                             cv=KFold(n_splits=20))
    print('Точность без параметров: {:.5f} ({:.5f})'.format(result.mean(), result.std()))

    # Количество деревьев (default=10)
    n_estimators = np.arange(60, 200, 20)
    for n_estimator in n_estimators:
        result = cross_val_score(RandomForestClassifier(n_estimators=n_estimator), 
                                 train_wine_attr, train_wine_quality,
                                 cv=KFold(n_splits=20))
        print('n_estimators =', n_estimator, result.mean(), result.std())
    print('--------------')

    # Функция оценки качества разбиения (default=”gini”)
    criterions = ['gini', 'entropy']
    for criterion in criterions:
        result = cross_val_score(RandomForestClassifier(criterion=criterion), 
                                 train_wine_attr, train_wine_quality,
                                 cv=KFold(n_splits=20))
        print('criterion =', criterion, result.mean(), result.std())
    print('--------------')

    # Количество фич при поиске наилучшего разбиения (default=”auto”)
    max_features = ['auto', 'log2', 1, 3, 5, 7, 9, 11]
    for features in max_features:
        result = cross_val_score(RandomForestClassifier(max_features=features), 
                                 train_wine_attr, train_wine_quality,
                                 cv=KFold(n_splits=20))
        print('max_features =', features, result.mean(), result.std())
    print('--------------')

    # Использовать ли предыдушие решения? (default=False)
    warm_start = [True, False]
    for war_start in warm_start:
        result = cross_val_score(RandomForestClassifier(warm_start=war_start), 
                                 train_wine_attr, train_wine_quality,
                                 cv=KFold(n_splits=20))
        print('warm_start =', war_start, result.mean(), result.std())
    print('--------------')
    
test_RandomForestClassifier(train_wine_attr, train_wine_quality)

Тестирование RandomForestClassifier


Точность без параметров: 0.59960 (0.02491)


n_estimators = 60 0.6442 0.0275818781086


n_estimators = 80 0.6388 0.0284843816854


n_estimators = 100 0.6474 0.0335922610135


...

max_features = 7 0.597 0.0262259413558


max_features = 9 0.5976 0.0344185996229


max_features = 11 0.5926 0.0231266080522
--------------


warm_start = True 0.5974 0.028207091307


warm_start = False 0.5908 0.018508376482
--------------


Изменение некоторых параметров классификаторов существенно увеличило точность определения качества. Определим оптимальный набор параметров каждого классификатора.

Метод ExtraTreesClassifier показал лучшие результаты классификации. Используем его для решения нашей задачи.

Метод ExtraTreesClassifier показал лучшие результаты классификации. Используем его для решения нашей задачи.

#### 3. Решение задачи

Для решения задачи нужно обучить выбранный классификатор и применить его к тестовой выборке. Результат будем выводить в файл в формате "id, quality".

In [16]:
def train():
    train_wine_attr, train_wine_quality, s = load_train_data()
    test_wine_attr, id = load_test_data(s)

    clf = ExtraTreesClassifier(n_estimators=50, criterion='entropy', max_features='log2', warm_start=True)
    clf.fit(train_wine_attr, train_wine_quality)
    result = clf.predict(test_wine_attr)

    # Создание выходного файла
    output = pd.DataFrame(id)
    output['quality'] = result
    output.to_csv('submission.csv', index=False)
    
train()

## 5. Заключение

В ходе выполнения работы была решена задача определения качества вина на основании только качественных характеристик напитка (без использования экспертных оценок). Для этого мы скачали и проанализировали данные с сайта kaggle.com, затем применили к нему наиболее точный алгоритм классификации из библиотеки scikit learn. Модель ExtraTreesClassifier показала самые высокие результаты на обучающей выборке. Результат классификации тестового датасета был сформирован в указанном в задании формате. Проверить точность классификации не удалось, так как выбранное соревнование относится к завершенным и ответы не принимаются.